In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'XXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX'
resource_group = 'ResourceGroupName'
workspace_name = 'AzureMLWorkspaceName'

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group )



In [2]:
from azureml.core.model import Model
# Here is where you specify location of your stored H2O model
model = Model.register(model_path="H2o/KMeans_model_python_1619773255297_1",
                          model_name="CustomerSegment",
                          description="Classify customer segment use K-Means Clustering",
                          workspace=ws)

Registering model CustomerSegment


In [3]:
from azureml.core import Environment
#Prepare Docker and requier dependency 
myenv = Environment(name="myenv")
myenv.docker.enabled = True

# Specify docker steps as a string.
dockerfile = r'''
FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04

# Install OpenJDK-8
RUN apt-get update && \
    apt-get install -y openjdk-8-jdk && \
    apt-get install -y ant && \
    apt-get clean;

# Fix certificate issues
RUN apt-get update && \
    apt-get install ca-certificates-java && \
    apt-get clean && \
    update-ca-certificates -f;

# Setup JAVA_HOME -- useful for docker commandline
ENV JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64/
RUN export JAVA_HOME

# Install H2O on python requirements
RUN pip install requests && \
    pip install tabulate && \
    pip install six && \
    pip install future && \
    pip install colorama

# Expose H2O Flow UI port
EXPOSE 54321

# Install H2O
RUN \
    pip uninstall h2o || true && \
    pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html --trusted-host h2o-release.s3.amazonaws.com h2o

'''

# Alternatively, load from a file.
#with open("dockerfiles/Dockerfile", "r") as f:
#    dockerfile=f.read()

myenv.docker.base_dockerfile = dockerfile



'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.
Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


In [4]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment

conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
#conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("pandas")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
# H2O version should be the same as the one you use when train model 
conda_dep.add_pip_package("h2o==3.32.0.4")
#conda_dep.add_pip_package("gensim")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep
myenv.inferencing_stack_version='latest'

for pip_package in ["inference_schema"]:
    myenv.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script="scoring.py",   
                                   environment=myenv)

In [102]:
from azureml.core.webservice import AciWebservice, Webservice
model = Model(ws, name='CustomerSegment')
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "customersegment", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI: http://5302b192-de92-471a-954b-c9f219efb43a.southeastasia.azurecontainer.io/score


In [98]:
#Test Calling Service 
X_df = pd.DataFrame(data=[{
    "CreditScore": 619,
    "Tenure": 2,
    "Balance": 0.00,
    "NumOfProducts": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 101348.88

}])

In [99]:
X_df.values.tolist()

[[0.0, 619.0, 101348.88, 1.0, 1.0, 2.0]]

In [103]:


import json

input_payload = json.dumps({
    'data': X_df.values.tolist()
 # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

[0]
